In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy
import requests
from Weather_Config import weather_api_key
from datetime import datetime

In [2]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=6faa1d1863e2f802e364d3f5294d3587


In [3]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [4]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [5]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

788

In [6]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

Beginning Data Retrieval     
-----------------------------


In [7]:
# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_desc = city_weather["weather"][0]["description"]
        # Convert the date to ISO standard.
        # city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": city_desc })

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

7 | marrakesh
Processing Record 38 of Set 7 | castro
Processing Record 39 of Set 7 | shawnee
Processing Record 40 of Set 7 | bahia honda
Processing Record 41 of Set 7 | meadow lake
Processing Record 42 of Set 7 | libres
Processing Record 43 of Set 7 | sungurlu
Processing Record 44 of Set 7 | hailey
Processing Record 45 of Set 7 | lolua
City not found. Skipping...
Processing Record 46 of Set 7 | ugoofaaru
Processing Record 47 of Set 7 | trairi
Processing Record 48 of Set 7 | hervey bay
Processing Record 49 of Set 7 | tevaitoa
Processing Record 50 of Set 7 | saldanha
Processing Record 1 of Set 8 | paamiut
Processing Record 2 of Set 8 | manakara
Processing Record 3 of Set 8 | gravdal
Processing Record 4 of Set 8 | diego de almagro
Processing Record 5 of Set 8 | mikhaylov
Processing Record 6 of Set 8 | lashio
Processing Record 7 of Set 8 | tromso
Processing Record 8 of Set 8 | palmer
Processing Record 9 of Set 8 | kautokeino
Processing Record 10 of Set 8 | hunza
City not found. Skipping...

In [8]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Rikitea,PF,-23.1203,-134.9692,70.09,67,100,26.28,overcast clouds
1,Saint-Augustin,CA,51.2260,-58.6502,62.31,66,21,2.35,few clouds
2,Samarai,PG,-10.6167,150.6667,80.67,72,77,14.34,broken clouds
3,Hilo,US,19.7297,-155.0900,73.26,94,94,5.75,light rain
4,Jamestown,US,42.0970,-79.2353,65.01,94,1,1.99,clear sky
5,Shangzhi,CN,45.2167,127.9667,88.41,56,32,7.25,scattered clouds
6,Kapaa,US,22.0752,-159.3190,80.92,78,90,5.01,light rain
7,Hithadhoo,MV,-0.6000,73.0833,81.82,68,57,0.81,broken clouds
8,Riyadh,SA,24.6877,46.7219,95.14,51,23,1.50,few clouds
9,Port Macquarie,AU,-31.4333,152.9167,65.89,52,29,9.10,scattered clouds


In [9]:
# Create the output file (CSV).
output_data_file = "WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")